In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
import yaml

import graphframes as gf
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Column

while not Path("data") in Path(".").iterdir():
    os.chdir("..")

plt.style.use("seaborn-white")
conf_dict = yaml.safe_load(Path("config/conf.yaml").read_text())

sc = SparkContext.getOrCreate()
sc.setLogLevel("ERROR")

spark = SparkSession.builder.config("spark.driver.memory", "8g").getOrCreate()

In [ ]:
companies_df = spark.read.parquet(conf_dict["companies_nodes"])
persons_df = spark.read.parquet(conf_dict["persons_nodes"])
edges_df = spark.read.parquet(conf_dict["edges"])

In [ ]:
companies_df.columns

In [ ]:
persons_df.columns

In [ ]:
edges_df.printSchema()

In [ ]:
print("node count:", nodes_df.count())
print("edge count:", edges_df.count())
print("component count:", nodes_df.select("component").distinct().count())